In [1]:
import boto3
from sfn_setup import make_def 
import json
import numpy as np

In [2]:
# Initialize boto3 handler
rds = boto3.client('rds')

# Describes your currently active databases
DBs = rds.describe_db_instances()['DBInstances']
activeDBs = [ x['DBName'] for x in DBs ]
activeDBs

['finalproject']

In [3]:
DB_NAME = 'finalproject'
INSTANCE_NAME = 'final-instance'

if DB_NAME not in activeDBs:
    # Create a new MySQL database on AWS
    response = rds.create_db_instance(
        DBInstanceIdentifier=INSTANCE_NAME,
        DBName=DB_NAME,
        MasterUsername='username',
        MasterUserPassword='password',
        DBInstanceClass='db.t3.micro',   # This specifies the EC2 instance that will run the datbase
        Engine='MySQL',
        AllocatedStorage=5
    )
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier=INSTANCE_NAME)

In [4]:
db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )

final-instance is available at final-instance.cjasscu8073h.us-east-1.rds.amazonaws.com on Port 3306


In [5]:
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust security permissions
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [5]:
import mysql.connector

# Connect to the MySQL database
conn =  mysql.connector.connect(host=ENDPOINT, 
                                user="username", 
                                passwd="password", 
                                port=PORT, 
                                database=DB_NAME)
cur = conn.cursor()

In [17]:
cur.execute(f"""
        CREATE TABLE IF NOT EXISTS simulation_data (
        combination_id VARCHAR(36),
        RunId INT NOT NULL,
        iteration INT NOT NULL,
        Step INT NOT NULL,
        num_tasks INT NOT NULL,
        num_nodes INT NOT NULL,
        num_new_edges INT NOT NULL,
        skills_proportion FLOAT NOT NULL,
        prob_memory FLOAT NOT NULL,
        availability FLOAT NOT NULL,
        time_lst TEXT NOT NULL,
        actor_sequence_lst TEXT NOT NULL 
    );
"""
)

In [44]:
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
aws_lambda = boto3.client('lambda')

lambda_arn = aws_lambda.get_function(FunctionName='routine_final')['Configuration']['FunctionArn']
sf_def = make_def(lambda_arn)

In [45]:
sfn = boto3.client('stepfunctions') 

try:
    response = sfn.create_state_machine(
        name='routine_final_sfn',
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn'],
        type='EXPRESS'
    )
except sfn.exceptions.StateMachineAlreadyExists:
    response = sfn.list_state_machines()
    state_machine_arn = [sm['stateMachineArn'] 
                         for sm in response['stateMachines'] 
                         if sm['name'] == 'routine_final_sfn'][0]
    response = sfn.update_state_machine(
        stateMachineArn=state_machine_arn,
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn']
    )

# Get arn for Step Function state machine
response = sfn.list_state_machines()
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'routine_final_sfn'][0]

In [46]:
parameters = {
    "num_tasks": 8,
    "num_nodes": list(range(10, 101, 10)),
    "num_new_edges": range(1, 9, 1),
    "skills_proportion": np.linspace(0.1, 0.5, 5),
    "prob_memory": list(np.linspace(0, 1.0, 6)),
    "availablity": 0.1         # list(np.linspace(0.1, 1.0, 4))
}

In [47]:
num_workers = 10
chunk_size = 50 // num_workers
parameters["iterations"] = chunk_size
data = [parameters]*num_workers

In [ ]:
response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='routine_final_sfn',
    input=json.dumps(data)
)

In [6]:
cur.execute('SELECT COUNT(*) FROM simulation_data;')
query_results = cur.fetchall()
print(query_results)

[(4000,)]
